# Projekt założenia
symulator zdarzeniowy M/M/1/N
    - średni czas oczekiwania na obsługę
    - prawdopobieństwo blokady
porównać zmodelem analitycznym:
    - dla N = 51
    - różnych wartości obciążenia: 0.01, 0.05, 0.1

średni czas oczekiwania - > histogram


In [155]:
# https://pyshark.com/poisson-distribution-and-poisson-process-in-python/#poisson-distribution-example-in-python
import scipy
import numpy as np
import matplotlib
from enum import Enum
import uuid

czas w serwerze to jest "usługa" 

In [160]:
# global_time = 0
n = 51 # max order in a system
# zmienna lososowa opisująca czas między zgłoszeniami
la = 3.35 # 1.0478 # 0.977 #  # intensywność napływu
# time_between_packet = 1 / la
# zmienna losowa opisuje czas pomiędzy zgłoszeniami k przez serwer
mu = 3
# time_of_service = 1 / mu # intensywność obsługi
time_of_simulation = 10
np.random.seed(12)

# https://github.com/amasl2048/queue-mm1/blob/master/qtest.cpp
roh = la / mu
rn = pow(roh, n + 1)
p0 = (1 - roh) / (1 - rn)         # free queue probability
pb = p0 * rn / roh                

navg = roh / (1-roh) - (n+1) * rn/(1-rn)  
tavg = navg / la                    
u    = mu * (1 - p0)                

out = '''
pb   = %.4f    # block probability
navg = %.4f    # average orders in a system
tavg = %.4f    # average time in a system
u    = %.4f    # throughput
'''
print(out % (pb, navg, tavg, u))


pb   = 0.1048    # block probability
navg = 42.5966    # average orders in a system
tavg = 12.7154    # average time in a system
u    = 2.9989    # throughput



In [157]:
class EventType(Enum):
    IN_EVENT = "IN_EVENT"
    OUT_EVENT = "OUT_EVENT"

# slajd 91, 

class Event():
    def __init__(self, time: float, event_type, client_id = None):
        self.time = time
        self.event_type = event_type
        if client_id == None:
            self.client_id = str(uuid.uuid4())
        else:
            self.client_id = client_id
    
    def __str__(self):
        mes = f"{self.client_id}, {self.event_type}, {self.time}"
        return mes
    
    def set_time_in_queue(self, time_in_queue):
        self.time_in_queue = time_in_queue
    
    def get_event_stats(self):
        return {
            'oc_time': self.time,
            'qu_time': self.time_in_queue,
            'client_id': self.client_id,
            'event_type': self.event_type
        }

In [181]:
class Server:
    def __init__(self):
        self.all_clients =0 
        self.all_clients_in_queue = 0
        self.clients_number_in_server = 0
        self.clients_number_in_queue = 0 
        self.clients_in_server = []]
        self.clients_in_queue = []
        self.buffor = n
    
    def get_server_stats(self):
        return self.clients_number_in_server, self.clients_number_in_queue

    def is_server_empty(self):
        return not self.clients_in_server

    def is_queue_empty(self):
        return not self.clients_in_queue

    def is_place_for_new_event(self):
        return self.clients_number_in_queue < self.buffor

    def add_client(self, client_id, timestamp):
        self.all_clients += 1
        self.clients_number_in_server += 1
        self.clients_in_server.insert(0, (client_id, timestamp))

    def add_client_to_queue(self, client_id, timestamp):
        self.all_clients_in_queue += 1
        self.clients_number_in_queue += 1
        self.clients_in_queue.insert(0, (client_id, timestamp))

    def get_queue_first_client(self):
        return self.clients_in_queue[-1]

    def delete_client(self, client_id):
        self.clients_number_in_server -= 1
        self.clients_in_server = list(
            filter(
                lambda client: client[0] != client_id,
                self.clients_in_server))

    def delete_client_from_queue(self, client_id):
        self.clients_number_in_queue -= 1
        self.clients_in_queue = list(
            filter(
                lambda client: client[0] != client_id,
                self.clients_in_queue))

    def clear_server(self):
        self.all_clients = 0
        self.all_clients_in_queue = 0
        self.clients_number_in_server = 0
        self.clients_number_in_queue = 0
        self.clients_in_server = []
        self.clients_in_queue = []
s = Server()
for i in range(53):
    s.add_client(str(i), str(i))
print(s.buffor)


51


In [159]:
# https://github.com/gsharma007/Stochastic_OR_and_Simulation_Modeling/blob/master/Queue_Modeling/DES.py
import logging

class Simulation:
    def __init__(self):
        self.num_in_system = 0
        self.clock = 0.0
        self.t_arrival = self.generate_interarrival()
        self.t_depart = float('inf') 
        self.num_arrivals = 0
        self.num_departs = 0
        self.total_wait = 0.0
        self.tmp_time = self.generate_service()
        # marge
        self.events = []
        self.clients_stats = []
        self.server = Server()
        self.clients = 0 
        # logger
         
         # a gdzie jest buffor? 

    def handle_event(self):
        current_event = self.events.pop()
        current_time = current_event.time
        # loggin
        if current_event.event_type == EventType.IN_EVENT:
            pass
            # make out event 
            # check buffor
            # 

    
    
    def generate_interarrival(self):
        return np.random.exponential(la)
    
    def generate_service(self):
        return np.random.exponential(mu)


    def advance_time(self):
        t_event = min(self.t_arrival, self.t_depart)
        self.total_wait += self.num_in_system*(t_event - self.clock)
        self.clock = t_event
        if self.t_arrival <= self.t_depart:
            self.handle_arrival_event()
        else:
            self.handle_depart_event()
    


    """
    def handle_arrival_event(self):
        self.num_in_system += 1
        self.num_arrivals += 1
        if self.num_in_system <= 1:
            self.temp1 = self.generate_service()
            # print(">>>>>>>>",self.temp1)
            self.tmp_time=self.temp1
            self.t_depart = self.clock + self.temp1
        self.t_arrival = self.clock + self.generate_interarrival()
        
    def handle_depart_event(self):
        self.num_in_system -= 1
        self.num_departs += 1
        if self.num_in_system > 0:
            self.temp2=self.generate_service();
            # print(">>>>><<<<<",self.temp2)
            self.tmp_time = self.temp2
            self.t_depart = self.clock + self.temp2
        else:
            self.t_depart = float("inf")
    
    # zmienieć na poissona

  

    def average_time(self):
        return self.total_wait / s.num_departs
    """


"""
s = Simulation()

clock_time = []
arrival_times = []
service_times = []
departures = []
event_calendar = []


for i in range(1000):
    s.advance_time()
    a = s.clock
    b = s.t_arrival
    c = s.tmp_time
    d = s.t_depart
    clock_time.append(a)
    arrival_times.append(b)
    service_times.append(c)
    departures.append(d)
    
    event_list = {"Iteration": "I_"+str(i), "Event_time": a,
                  "Arrival_time": b, "Service_time": c, "Departure_time": d}
    event_calendar.append(event_list)
    print(event_list)

print(s.average_time()) 
"""

{'Iteration': 'I_0', 'Event_time': 0.5608852170027645, 'Arrival_time': 3.1169705913299692, 'Service_time': 0.9167847215492568, 'Departure_time': 1.4776699385520213}
{'Iteration': 'I_1', 'Event_time': 1.4776699385520213, 'Arrival_time': 3.1169705913299692, 'Service_time': 0.9167847215492568, 'Departure_time': inf}
{'Iteration': 'I_2', 'Event_time': 3.1169705913299692, 'Arrival_time': 11.526099168795117, 'Service_time': 0.04404666215009944, 'Departure_time': 3.1610172534800687}
{'Iteration': 'I_3', 'Event_time': 3.1610172534800687, 'Arrival_time': 11.526099168795117, 'Service_time': 0.04404666215009944, 'Departure_time': inf}
{'Iteration': 'I_4', 'Event_time': 11.526099168795117, 'Arrival_time': 11.639974673111745, 'Service_time': 6.92927792222578, 'Departure_time': 18.455377091020896}
{'Iteration': 'I_5', 'Event_time': 11.639974673111745, 'Arrival_time': 22.176989735930658, 'Service_time': 6.92927792222578, 'Departure_time': 18.455377091020896}
{'Iteration': 'I_6', 'Event_time': 18.4553

get pobiera zdarzenie z listy z namnijeszym czasem, przesuwa czas symulacji do tego czasu

To do: 
- ogarnać czy potrzebny jest server
    - liczba n ?
- dodac logging do servera lub klientów
- zmienić symualcje od maksymalnego czasu, a nie iteracji 
- zrobić jakieś wykresy od lambda ? 
- zrobić historgram dla b
- sprawko 
- wysłać

UAKTUALNIENIE: ponieważ w punkcie III projektu wkradł się błąd a jest już blisko terminu oddania prac, proszę wykonać tę część przy prostym założeniu, że odstęp miedzy przychodzącymi klientami jest stały i równy średniej z punktu I. 
Reference materials